# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    # join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root, '*'))
# print(file_path_list)


/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = []
# for every filepath in the file path list
for f in file_path_list:
    # reading csv file
    with open(f, 'r', encoding='utf8', newline='') as csvfile:
        # creating a csv reader object
        csvreader = csv.reader(csvfile)
        next(csvreader)
        # extracting each data row one by one and append it
        for line in csvreader:
            # print(line)
            full_data_rows_list.append(line)
# creating a smaller event data csv file called event_datafile_full csv
# that will be used to insert data into the
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding='utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist', 'firstName', 'gender', 'itemInSession',
                     'lastName', 'length', 'level', 'location', 'sessionId',
                     'song', 'userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6],
                         row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding='utf8') as f:
    print(sum(1 for line in f))


6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])

# To establish connection and begin executing queries, need a session
session = cluster.connect()


#### Create Keyspace

In [6]:
# TO-DO: Create a Keyspace
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }
    """)
except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
# TO-DO: Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('udacity')
    
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [9]:
query = "CREATE TABLE IF NOT EXISTS Artist_Song_by_sessionId"
query = query + "(sessionId int, itemInSession int, artist text, song varchar,  \
                  length float, primary key ((sessionId, itemInSession)))"

try:
    session.execute(query)
except Exception as e:
    print(e)


In [10]:
# We have provided part of the code to set up the CSV file. Please complete
# the Apache Cassandra code below
file = 'event_datafile_new.csv'

with open(file, encoding='utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)  # skip header
    for line in csvreader:
        # TO-DO: Assign the INSERT statements into the `query` variable
        query = "INSERT INTO Artist_Song_by_sessionId (sessionId, \
                 itemInSession, artist, song, length)"
        query = query + " VALUES (%s, %s, %s, %s ,%s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))


#### Do a SELECT to verify that the data have been inserted into each table

In [11]:
# Query Description: In this query, I used sessionid and iteminsession \
# as the partition key. The combination of both uniquely identifies \
# the rows, since some rows got same sessionid but are uniquely \
# identified by iteminsession.
query = "select artist, song, length from Artist_Song_by_sessionId \
        where itemInSession=4 and sessionId=338"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
for row in rows:
    print (row.artist, "|", row.song, "|", row.length)


Faithless | Music Matters (Mark Knight Dub) | 495.30731201171875


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [12]:
query = "CREATE TABLE IF NOT EXISTS Song_info_Sorted_byuserid_sessionid"
query = query + "(userid int, sessionid int, iteminsession int, \
                  artist text, song varchar, firstname text, lastname text, \
                  primary key ((userid, sessionid),iteminsession))"
try:
    session.execute(query)
except Exception as e:
    print(e)


In [13]:
# We have provided part of the code to set up the CSV file. Please complete
# the Apache Cassandra code below#
file = 'event_datafile_new.csv'
with open(file, encoding='utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)  # skip header
    for line in csvreader:
        # TO-DO: Assign the INSERT statements into the `query` variable
        query = "INSERT INTO Song_info_Sorted_byuserid_sessionid \
                  (userid, sessionid, iteminsession, artist, song, \
                  firstname, lastname)"
        query = query + " VALUES (%s, %s, %s, %s ,%s , %s ,%s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0],
                                line[9], line[1], line[4]))


In [14]:
# Query Description: In this query, I used userid or sessionid as \
# partition key cause i am filtering by them and since we need to sort by \
# iteminsession i used it as my clustering key.

query = "select artist, song, firstname, lastname, \
         sessionid, userid, iteminsession \
         from Song_info_Sorted_byuserid_sessionid \
         where userid=10 and sessionid=182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
for row in rows:
    print (row.artist, "|", row.song, "|", row.firstname, "|", row.lastname,
           "|", row.sessionid, "|", row.userid)


Down To The Bone | Keep On Keepin' On | Sylvie | Cruz | 182 | 10
Three Drives | Greece 2000 | Sylvie | Cruz | 182 | 10
Sebastien Tellier | Kilometer | Sylvie | Cruz | 182 | 10
Lonnie Gordon | Catch You Baby (Steve Pitron & Max Sanna Radio Edit) | Sylvie | Cruz | 182 | 10


In [15]:
query = "CREATE TABLE IF NOT EXISTS FullName_Sorted_bysong"
query = query + "(song varchar, firstname text, lastname text, \
                  iteminsession int, primary key ((song),iteminsession))"
try:
    session.execute(query)
except Exception as e:
    print(e)


In [16]:
# We have provided part of the code to set up the CSV file.
# Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'
with open(file, encoding='utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)  # skip header
    for line in csvreader:
        # TO-DO: Assign the INSERT statements into the `query` variable
        query = "INSERT INTO FullName_Sorted_bysong (song, firstname, lastname, iteminsession)"
        query = query + " VALUES (%s, %s, %s, %s
        session.execute(query, (line[9], line[1], line[4], int(line[3])))


In [17]:
# Query Description: In this query, Since i am filtring by song i will use \
# it as partition key, as for uniquly sorted rows i will use iteminsession
query = "select firstname, lastname, song, iteminsession \
         from FullName_Sorted_bysong \
         where song='All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
for row in rows:
    print(row.firstname, "|", row.lastname, "|", row.song,
          "|", row.iteminsession)


Tegan | Levine | All Hands Against His Own | 25
Sara | Johnson | All Hands Against His Own | 31
Jacqueline | Lynch | All Hands Against His Own | 50


### Drop the tables before closing out the sessions

In [18]:
query = "drop table Artist_Song_by_sessionId"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)


In [19]:
query = "drop table Song_info_Sorted_byuserid_sessionid"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)


In [20]:
query = "drop table FullName_Sorted_bysong"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)


### Close the session and cluster connection¶

In [21]:
session.shutdown()
cluster.shutdown()
